# Appendix 2: Feature Selection Process

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

pd.options.display.max_columns = None
pd.set_option('display.float_format', '{:.4f}'.format)

In [2]:
data_path = 'home-credit-default-risk/'

In [3]:
def describe_df(df):
    print(df.shape)
    
    desc = df.describe()
    display(desc)
    
    if desc.shape[1] == df.shape[1]:
        return
    
    desc_o = df.describe(include='object')
    display(desc_o)

    for c in desc_o.columns:
        if df[c].nunique() < 20:
            print(df[c].fillna('-- null --').value_counts(normalize=True))
            print()
            
def clean_columns_name(df, prefix=''):
    return [(prefix+'_'.join(col)).upper().replace(' ', '_') for col in df.columns]

In [4]:
df_app_train = pd.read_csv(data_path + 'application_train.csv')
df_app_test = pd.read_csv(data_path + 'application_test.csv')

## Step 1: Application features only as baseline model

In [6]:
import shap
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

def prepare_train_test_set(df_train, df_test):
    if 'SK_ID_CURR' in df_train.columns:
        X_train = df_train.drop(['SK_ID_CURR'], axis=1)
        X_test = df_test.drop(['SK_ID_CURR'], axis=1)
    else:
        X_train = df_train.copy()
        X_test = df_test.copy()
    y_train = df_app_train['TARGET']

    ## to handle missing dummy columns 
    to_add_cols = [c for c in X_train.columns if c not in X_test.columns]
    for c in to_add_cols:
        X_test[c] = 0
    X_test = X_test[X_train.columns]
    return X_train, y_train, X_test

def fit_lgb(X_train, y_train, X_test):
    folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)
    oof_preds = np.zeros(X_train.shape[0])
    test_preds = np.zeros(X_test.shape[0])
    
    for train_idx, val_idx in folds.split(X_train, y_train):
        X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]

        model = lgb.LGBMClassifier(
            n_estimators=1000, learning_rate=0.05,
            early_stopping_round=50,
            random_state=123, force_col_wise=True
        )
        model.fit(X_tr, y_tr, eval_set=[(X_val, y_val)], eval_metric='auc')

        oof_preds[val_idx] = model.predict_proba(X_val)[:, 1]
        test_preds += model.predict_proba(X_test)[:, 1] / folds.n_splits
        
    return model, oof_preds, test_preds

def cal_shap(model, X):
    explainer = shap.Explainer(model)
    shap_values = explainer(X)
    return shap_values

def cal_top_features(shap_values, X):
    shap_importance = np.abs(shap_values[:, :, 1].values).mean(axis=0)
    features = pd.Series(shap_importance, index=X.columns).sort_values(ascending=False)
    top_features = features[features > 0.01]
    print('# Top features:', len(top_features))
    print(top_features.head(50))
    return features

def get_shap_features(model, X):
    shap_values = cal_shap(model, X)
    features = cal_top_features(shap_values, X)
    return shap_values, features

In [7]:
## identify fill type based on feature meaning

to_fill_zero_cols = [
    'AMT_ANNUITY', 'AMT_GOODS_PRICE', 'OWN_CAR_AGE', 'CNT_FAM_MEMBERS', 'DAYS_LAST_PHONE_CHANGE',
    ## How many observation of client's social surroundings
    'OBS_30_CNT_SOCIAL_CIRCLE', 'DEF_30_CNT_SOCIAL_CIRCLE', 'OBS_60_CNT_SOCIAL_CIRCLE', 'DEF_60_CNT_SOCIAL_CIRCLE',
    ## Number of enquiries to Credit Bureau about the client
    'AMT_REQ_CREDIT_BUREAU_HOUR', 'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_WEEK',
    'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_QRT', 'AMT_REQ_CREDIT_BUREAU_YEAR'
]

to_fill_avg_cols = [
    ## Normalized score from external data source
    'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3',
    ## Normalized information about building where the client lives
    'APARTMENTS_AVG', 'BASEMENTAREA_AVG',
    'YEARS_BEGINEXPLUATATION_AVG', 'YEARS_BUILD_AVG', 'COMMONAREA_AVG',
    'ELEVATORS_AVG', 'ENTRANCES_AVG', 'FLOORSMAX_AVG', 'FLOORSMIN_AVG',
    'LANDAREA_AVG', 'LIVINGAPARTMENTS_AVG', 'LIVINGAREA_AVG',
    'NONLIVINGAPARTMENTS_AVG', 'NONLIVINGAREA_AVG', 'APARTMENTS_MODE',
    'BASEMENTAREA_MODE', 'YEARS_BEGINEXPLUATATION_MODE', 'YEARS_BUILD_MODE',
    'COMMONAREA_MODE', 'ELEVATORS_MODE', 'ENTRANCES_MODE', 'FLOORSMAX_MODE',
    'FLOORSMIN_MODE', 'LANDAREA_MODE', 'LIVINGAPARTMENTS_MODE',
    'LIVINGAREA_MODE', 'NONLIVINGAPARTMENTS_MODE', 'NONLIVINGAREA_MODE',
    'APARTMENTS_MEDI', 'BASEMENTAREA_MEDI', 'YEARS_BEGINEXPLUATATION_MEDI',
    'YEARS_BUILD_MEDI', 'COMMONAREA_MEDI', 'ELEVATORS_MEDI',
    'ENTRANCES_MEDI', 'FLOORSMAX_MEDI', 'FLOORSMIN_MEDI', 'LANDAREA_MEDI',
    'LIVINGAPARTMENTS_MEDI', 'LIVINGAREA_MEDI', 'NONLIVINGAPARTMENTS_MEDI',
    'NONLIVINGAREA_MEDI', 'TOTALAREA_MODE'
]

In [11]:
def gen_feature_app(df_raw, to_fill_zero_cols=[], to_fill_avg_cols=[]):
    df_object = df_raw.select_dtypes(include='object')
    
    df = df_raw.copy()
    df[df_object.columns] = df_object.astype('category')
    df[to_fill_zero_cols] = df_app_train[to_fill_zero_cols].fillna(0)
    
    ## this method might be forward-looking bias for cross valiation, using for exploration only
    df[to_fill_avg_cols] = df_app_train[to_fill_avg_cols].fillna(df_app_train[to_fill_avg_cols].mean())
    return df

In [12]:
df_train = gen_feature_app(df_app_train)
df_test = gen_feature_app(df_app_test)

In [14]:
X_train, y_train, X_test = prepare_train_test_set(df_train.drop(['TARGET'], axis=1), df_test)
model, oof_preds, test_preds = fit_lgb(X_train, y_train, X_test)

print('CV AUC:', roc_auc_score(y_train, oof_preds))

[LightGBM] [Info] Number of positive: 19860, number of negative: 226148
[LightGBM] [Info] Total Bins 11369
[LightGBM] [Info] Number of data points in the train set: 246008, number of used features: 116
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.080729 -> initscore=-2.432482
[LightGBM] [Info] Start training from score -2.432482
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[356]	valid_0's auc: 0.759637	valid_0's binary_logloss: 0.245857
[LightGBM] [Info] Number of positive: 19860, number of negative: 226149
[LightGBM] [Info] Total Bins 11310
[LightGBM] [Info] Number of data points in the train set: 246009, number of used features: 115
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.080729 -> initscore=-2.432486
[LightGBM] [Info] Start training from score -2.432486
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[241]	valid_0's auc: 0.758195	valid_0's binary_logloss: 0.245879
[Light

## Step 2: Explore features

Try to create a lot of aggregated features

### 2.1) Bureau-related features

In [16]:
df_bureau = pd.read_csv(data_path + 'bureau.csv')
df_bureau

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0000,-153.0000,NaN,0,91323.0000,0.0000,NaN,0.0000,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0000,NaN,NaN,0,225000.0000,171342.0000,NaN,0.0000,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0000,NaN,NaN,0,464323.5000,NaN,NaN,0.0000,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0000,NaN,NaN,0.0000,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0000,NaN,77674.5000,0,2700000.0000,NaN,NaN,0.0000,Consumer credit,-21,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1716423,259355,5057750,Active,currency 1,-44,0,-30.0000,NaN,0.0000,0,11250.0000,11250.0000,0.0000,0.0000,Microloan,-19,NaN
1716424,100044,5057754,Closed,currency 1,-2648,0,-2433.0000,-2493.0000,5476.5000,0,38130.8400,0.0000,0.0000,0.0000,Consumer credit,-2493,NaN
1716425,100044,5057762,Closed,currency 1,-1809,0,-1628.0000,-970.0000,NaN,0,15570.0000,NaN,NaN,0.0000,Consumer credit,-967,NaN
1716426,246829,5057770,Closed,currency 1,-1878,0,-1513.0000,-1513.0000,NaN,0,36000.0000,0.0000,0.0000,0.0000,Consumer credit,-1508,NaN


In [17]:
describe_df(df_bureau)

(1716428, 17)


,SK_ID_CURR,SK_ID_BUREAU,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
count,1716428.0000,1716428.0000,1716428.0000,1716428.0000,1610875.0000,1082775.0000,591940.0000,1716428.0000,1716415.0000,1458759.0000,1124648.0000,1716428.0000,1716428.0000,489637.0000
mean,278214.9336,5924434.4890,-1142.1077,0.8182,510.5174,-1017.4371,3825.4177,0.0064,354994.5919,137085.1200,6229.5150,37.9128,-593.7483,15712.7577
std,102938.5581,532265.7286,795.1649,36.5444,4994.2198,714.0106,206031.6062,0.0962,1149811.3440,677401.1310,45032.0315,5937.6500,720.7473,325826.9491
min,100001.0000,5000000.0000,-2922.0000,0.0000,-42060.0000,-42023.0000,0.0000,0.0000,0.0000,-4705600.3200,-586406.1150,0.0000,-41947.0000,0.0000
25%,188866.7500,5463953.7500,-1666.0000,0.0000,-1138.0000,-1489.0000,0.0000,0.0000,51300.0000,0.0000,0.0000,0.0000,-908.0000,0.0000
50%,278055.0000,5926303.5000,-987.0000,0.0000,-330.0000,-897.0000,0.0000,0.0000,125518.5000,0.0000,0.0000,0.0000,-395.0000,0.0000
75%,367426.0000,6385681.2500,-474.0000,0.0000,474.0000,-425.0000,0.0000,0.0000,315000.0000,40153.5000,0.0000,0.0000,-33.0000,13500.0000
max,456255.0000,6843457.0000,0.0000,2792.0000,31199.0000,0.0000,115987185.0000,9.0000,585000000.0000,170100000.0000,4705600.3200,3756681.0000,372.0000,118453423.5000


,CREDIT_ACTIVE,CREDIT_CURRENCY,CREDIT_TYPE
count,1716428,1716428,1716428
unique,4,4,15
top,Closed,currency 1,Consumer credit
freq,1079273,1715020,1251615


CREDIT_ACTIVE
Closed     0.6288
Active     0.3674
Sold       0.0038
Bad debt   0.0000
Name: proportion, dtype: float64

CREDIT_CURRENCY
currency 1   0.9992
currency 2   0.0007
currency 3   0.0001
currency 4   0.0000
Name: proportion, dtype: float64

CREDIT_TYPE
Consumer credit                                0.7292
Credit card                                    0.2343
Car loan                                       0.0161
Mortgage                                       0.0107
Microloan                                      0.0072
Loan for business development                  0.0012
Another type of loan                           0.0006
Unknown type of loan                           0.0003
Loan for working capital replenishment         0.0003
Cash loan (non-earmarked)                      0.0000
Real estate loan                               0.0000
Loan for the purchase of equipment             0.0000
Loan for purchase of shares (margin lending)   0.0000
Mobile operator loan                

In [18]:
df_bureau_bal = pd.read_csv(data_path + 'bureau_balance.csv')
df_bureau_bal

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS
0,5715448,0,C
1,5715448,-1,C
2,5715448,-2,C
3,5715448,-3,C
4,5715448,-4,C
...,...,...,...
27299920,5041336,-47,X
27299921,5041336,-48,X
27299922,5041336,-49,X
27299923,5041336,-50,X


In [19]:
describe_df(df_bureau_bal)

(27299925, 3)


,SK_ID_BUREAU,MONTHS_BALANCE
count,27299925.0000,27299925.0000
mean,6036297.3330,-30.7417
std,492348.8569,23.8645
min,5001709.0000,-96.0000
25%,5730933.0000,-46.0000
50%,6070821.0000,-25.0000
75%,6431951.0000,-11.0000
max,6842888.0000,0.0000


,STATUS
count,27299925
unique,8
top,C
freq,13646993


STATUS
C   0.4999
0   0.2747
X   0.2128
1   0.0089
5   0.0023
2   0.0009
3   0.0003
4   0.0002
Name: proportion, dtype: float64



In [23]:
df_bb_agg = (
    df_bureau_bal
    .groupby('SK_ID_BUREAU')['MONTHS_BALANCE']
    .agg({'max', 'min', 'count'})
    .add_prefix('MONTHS_')
)

df_bb_status = (
    df_bureau_bal
    .pivot_table(
        index='SK_ID_BUREAU',
        columns='STATUS',
        values='MONTHS_BALANCE',
        aggfunc='count',
        fill_value=0
    )
    .add_prefix('MONTHS_STATUS_')
)

df_bureau_agg = (
    df_bureau
    .merge(df_bb_agg, on='SK_ID_BUREAU', how='left')
    .merge(df_bb_status, on='SK_ID_BUREAU', how='left')
)
print(df_bureau_bal.shape, '-->', df_bb_agg.shape, df_bb_status.shape)
print(df_bureau.shape, '-->', df_bureau_agg.shape)

(27299925, 3) --> (817395, 3) (817395, 8)
(1716428, 17) --> (1716428, 28)


In [21]:
describe_df(df_bureau_agg)

(1716428, 28)


,SK_ID_CURR,SK_ID_BUREAU,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,MONTHS_count,MONTHS_max,MONTHS_min,MONTHS_STATUS_0,MONTHS_STATUS_1,MONTHS_STATUS_2,MONTHS_STATUS_3,MONTHS_STATUS_4,MONTHS_STATUS_5,MONTHS_STATUS_C,MONTHS_STATUS_X
count,1716428.0000,1716428.0000,1716428.0000,1716428.0000,1610875.0000,1082775.0000,591940.0000,1716428.0000,1716415.0000,1458759.0000,1124648.0000,1716428.0000,1716428.0000,489637.0000,774354.0000,774354.0000,774354.0000,774354.0000,774354.0000,774354.0000,774354.0000,774354.0000,774354.0000,774354.0000,774354.0000
mean,278214.9336,5924434.4890,-1142.1077,0.8182,510.5174,-1017.4371,3825.4177,0.0064,354994.5919,137085.1200,6229.5150,37.9128,-593.7483,15712.7577,31.2257,-6.0630,-36.2887,9.2920,0.2967,0.0271,0.0101,0.0065,0.0650,14.9227,6.6056
std,102938.5581,532265.7286,795.1649,36.5444,4994.2198,714.0106,206031.6062,0.0962,1149811.3440,677401.1310,45032.0315,5937.6500,720.7473,325826.9491,23.4838,15.0643,25.4497,10.0841,1.2786,0.2634,0.1359,0.0990,1.3543,20.7232,14.6535
min,100001.0000,5000000.0000,-2922.0000,0.0000,-42060.0000,-42023.0000,0.0000,0.0000,0.0000,-4705600.3200,-586406.1150,0.0000,-41947.0000,0.0000,1.0000,-95.0000,-96.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
25%,188866.7500,5463953.7500,-1666.0000,0.0000,-1138.0000,-1489.0000,0.0000,0.0000,51300.0000,0.0000,0.0000,0.0000,-908.0000,0.0000,12.0000,0.0000,-53.0000,3.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
50%,278055.0000,5926303.5000,-987.0000,0.0000,-330.0000,-897.0000,0.0000,0.0000,125518.5000,0.0000,0.0000,0.0000,-395.0000,0.0000,25.0000,0.0000,-31.0000,7.0000,0.0000,0.0000,0.0000,0.0000,0.0000,4.0000,1.0000
75%,367426.0000,6385681.2500,-474.0000,0.0000,474.0000,-425.0000,0.0000,0.0000,315000.0000,40153.5000,0.0000,0.0000,-33.0000,13500.0000,45.0000,0.0000,-15.0000,12.0000,0.0000,0.0000,0.0000,0.0000,0.0000,25.0000,5.0000
max,456255.0000,6843457.0000,0.0000,2792.0000,31199.0000,0.0000,115987185.0000,9.0000,585000000.0000,170100000.0000,4705600.3200,3756681.0000,372.0000,118453423.5000,97.0000,0.0000,0.0000,95.0000,56.0000,33.0000,18.0000,11.0000,89.0000,96.0000,97.0000


,CREDIT_ACTIVE,CREDIT_CURRENCY,CREDIT_TYPE
count,1716428,1716428,1716428
unique,4,4,15
top,Closed,currency 1,Consumer credit
freq,1079273,1715020,1251615


CREDIT_ACTIVE
Closed     0.6288
Active     0.3674
Sold       0.0038
Bad debt   0.0000
Name: proportion, dtype: float64

CREDIT_CURRENCY
currency 1   0.9992
currency 2   0.0007
currency 3   0.0001
currency 4   0.0000
Name: proportion, dtype: float64

CREDIT_TYPE
Consumer credit                                0.7292
Credit card                                    0.2343
Car loan                                       0.0161
Mortgage                                       0.0107
Microloan                                      0.0072
Loan for business development                  0.0012
Another type of loan                           0.0006
Unknown type of loan                           0.0003
Loan for working capital replenishment         0.0003
Cash loan (non-earmarked)                      0.0000
Real estate loan                               0.0000
Loan for the purchase of equipment             0.0000
Loan for purchase of shares (margin lending)   0.0000
Mobile operator loan                

In [30]:
df_bureau_active = (
    df_bureau_agg
    .pivot_table(
        index='SK_ID_CURR',
        columns='CREDIT_ACTIVE',
        aggfunc={
            'SK_ID_BUREAU': 'count',
            'CREDIT_DAY_OVERDUE': 'max',
            'CNT_CREDIT_PROLONG': 'sum',
            **{c: ['sum', 'max', 'mean'] for c in df_bureau_agg.columns if c[:4] == 'AMT_'},
            'MONTHS_max': 'max',
            'MONTHS_min': 'min',
            'MONTHS_count': 'sum',
            **{c: 'sum' for c in df_bureau_agg.columns if '_count' in c or 'MONTHS_STATUS_' in c},
        },
        fill_value=0
    )
)
df_bureau_active.columns = clean_columns_name(df_bureau_active, 'BR_')

list_credit_type_main = ['Consumer credit', 'Credit card', 'Car loan', 'Mortgage']
df_bureau_agg.loc[df_bureau_agg['CREDIT_TYPE'].isin(list_credit_type_main), 'CREDIT_TYPE_MAIN'] = df_bureau_agg['CREDIT_TYPE']
df_bureau_type = (
    df_bureau_agg
    .pivot_table(
        index='SK_ID_CURR',
        columns='CREDIT_TYPE_MAIN',
        values=['SK_ID_BUREAU'],
        aggfunc=['count'],
        fill_value=0
    )
)
df_bureau_type.columns = clean_columns_name(df_bureau_type, 'BR_TYPE_')

In [35]:
describe_df(df_bureau_active)

(305811, 128)


,BR_AMT_ANNUITY_MAX_ACTIVE,BR_AMT_ANNUITY_MAX_BAD_DEBT,BR_AMT_ANNUITY_MAX_CLOSED,BR_AMT_ANNUITY_MAX_SOLD,BR_AMT_ANNUITY_MEAN_ACTIVE,BR_AMT_ANNUITY_MEAN_BAD_DEBT,BR_AMT_ANNUITY_MEAN_CLOSED,BR_AMT_ANNUITY_MEAN_SOLD,BR_AMT_ANNUITY_SUM_ACTIVE,BR_AMT_ANNUITY_SUM_BAD_DEBT,BR_AMT_ANNUITY_SUM_CLOSED,BR_AMT_ANNUITY_SUM_SOLD,BR_AMT_CREDIT_MAX_OVERDUE_MAX_ACTIVE,BR_AMT_CREDIT_MAX_OVERDUE_MAX_BAD_DEBT,BR_AMT_CREDIT_MAX_OVERDUE_MAX_CLOSED,BR_AMT_CREDIT_MAX_OVERDUE_MAX_SOLD,BR_AMT_CREDIT_MAX_OVERDUE_MEAN_ACTIVE,BR_AMT_CREDIT_MAX_OVERDUE_MEAN_BAD_DEBT,BR_AMT_CREDIT_MAX_OVERDUE_MEAN_CLOSED,BR_AMT_CREDIT_MAX_OVERDUE_MEAN_SOLD,BR_AMT_CREDIT_MAX_OVERDUE_SUM_ACTIVE,BR_AMT_CREDIT_MAX_OVERDUE_SUM_BAD_DEBT,BR_AMT_CREDIT_MAX_OVERDUE_SUM_CLOSED,BR_AMT_CREDIT_MAX_OVERDUE_SUM_SOLD,BR_AMT_CREDIT_SUM_MAX_ACTIVE,BR_AMT_CREDIT_SUM_MAX_BAD_DEBT,BR_AMT_CREDIT_SUM_MAX_CLOSED,BR_AMT_CREDIT_SUM_MAX_SOLD,BR_AMT_CREDIT_SUM_MEAN_ACTIVE,BR_AMT_CREDIT_SUM_MEAN_BAD_DEBT,BR_AMT_CREDIT_SUM_MEAN_CLOSED,BR_AMT_CREDIT_SUM_MEAN_SOLD,BR_AMT_CREDIT_SUM_SUM_ACTIVE,BR_AMT_CREDIT_SUM_SUM_BAD_DEBT,BR_AMT_CREDIT_SUM_SUM_CLOSED,BR_AMT_CREDIT_SUM_SUM_SOLD,BR_AMT_CREDIT_SUM_DEBT_MAX_ACTIVE,BR_AMT_CREDIT_SUM_DEBT_MAX_BAD_DEBT,BR_AMT_CREDIT_SUM_DEBT_MAX_CLOSED,BR_AMT_CREDIT_SUM_DEBT_MAX_SOLD,BR_AMT_CREDIT_SUM_DEBT_MEAN_ACTIVE,BR_AMT_CREDIT_SUM_DEBT_MEAN_BAD_DEBT,BR_AMT_CREDIT_SUM_DEBT_MEAN_CLOSED,BR_AMT_CREDIT_SUM_DEBT_MEAN_SOLD,BR_AMT_CREDIT_SUM_DEBT_SUM_ACTIVE,BR_AMT_CREDIT_SUM_DEBT_SUM_BAD_DEBT,BR_AMT_CREDIT_SUM_DEBT_SUM_CLOSED,BR_AMT_CREDIT_SUM_DEBT_SUM_SOLD,BR_AMT_CREDIT_SUM_LIMIT_MAX_ACTIVE,BR_AMT_CREDIT_SUM_LIMIT_MAX_BAD_DEBT,BR_AMT_CREDIT_SUM_LIMIT_MAX_CLOSED,BR_AMT_CREDIT_SUM_LIMIT_MAX_SOLD,BR_AMT_CREDIT_SUM_LIMIT_MEAN_ACTIVE,BR_AMT_CREDIT_SUM_LIMIT_MEAN_BAD_DEBT,BR_AMT_CREDIT_SUM_LIMIT_MEAN_CLOSED,BR_AMT_CREDIT_SUM_LIMIT_MEAN_SOLD,BR_AMT_CREDIT_SUM_LIMIT_SUM_ACTIVE,BR_AMT_CREDIT_SUM_LIMIT_SUM_BAD_DEBT,BR_AMT_CREDIT_SUM_LIMIT_SUM_CLOSED,BR_AMT_CREDIT_SUM_LIMIT_SUM_SOLD,BR_AMT_CREDIT_SUM_OVERDUE_MAX_ACTIVE,BR_AMT_CREDIT_SUM_OVERDUE_MAX_BAD_DEBT,BR_AMT_CREDIT_SUM_OVERDUE_MAX_CLOSED,BR_AMT_CREDIT_SUM_OVERDUE_MAX_SOLD,BR_AMT_CREDIT_SUM_OVERDUE_MEAN_ACTIVE,BR_AMT_CREDIT_SUM_OVERDUE_MEAN_BAD_DEBT,BR_AMT_CREDIT_SUM_OVERDUE_MEAN_CLOSED,BR_AMT_CREDIT_SUM_OVERDUE_MEAN_SOLD,BR_AMT_CREDIT_SUM_OVERDUE_SUM_ACTIVE,BR_AMT_CREDIT_SUM_OVERDUE_SUM_BAD_DEBT,BR_AMT_CREDIT_SUM_OVERDUE_SUM_CLOSED,BR_AMT_CREDIT_SUM_OVERDUE_SUM_SOLD,BR_CNT_CREDIT_PROLONG_SUM_ACTIVE,BR_CNT_CREDIT_PROLONG_SUM_BAD_DEBT,BR_CNT_CREDIT_PROLONG_SUM_CLOSED,BR_CNT_CREDIT_PROLONG_SUM_SOLD,BR_CREDIT_DAY_OVERDUE_MAX_ACTIVE,BR_CREDIT_DAY_OVERDUE_MAX_BAD_DEBT,BR_CREDIT_DAY_OVERDUE_MAX_CLOSED,BR_CREDIT_DAY_OVERDUE_MAX_SOLD,BR_MONTHS_STATUS_0_SUM_ACTIVE,BR_MONTHS_STATUS_0_SUM_BAD_DEBT,BR_MONTHS_STATUS_0_SUM_CLOSED,BR_MONTHS_STATUS_0_SUM_SOLD,BR_MONTHS_STATUS_1_SUM_ACTIVE,BR_MONTHS_STATUS_1_SUM_BAD_DEBT,BR_MONTHS_STATUS_1_SUM_CLOSED,BR_MONTHS_STATUS_1_SUM_SOLD,BR_MONTHS_STATUS_2_SUM_ACTIVE,BR_MONTHS_STATUS_2_SUM_BAD_DEBT,BR_MONTHS_STATUS_2_SUM_CLOSED,BR_MONTHS_STATUS_2_SUM_SOLD,BR_MONTHS_STATUS_3_SUM_ACTIVE,BR_MONTHS_STATUS_3_SUM_BAD_DEBT,BR_MONTHS_STATUS_3_SUM_CLOSED,BR_MONTHS_STATUS_3_SUM_SOLD,BR_MONTHS_STATUS_4_SUM_ACTIVE,BR_MONTHS_STATUS_4_SUM_BAD_DEBT,BR_MONTHS_STATUS_4_SUM_CLOSED,BR_MONTHS_STATUS_4_SUM_SOLD,BR_MONTHS_STATUS_5_SUM_ACTIVE,BR_MONTHS_STATUS_5_SUM_BAD_DEBT,BR_MONTHS_STATUS_5_SUM_CLOSED,BR_MONTHS_STATUS_5_SUM_SOLD,BR_MONTHS_STATUS_C_SUM_ACTIVE,BR_MONTHS_STATUS_C_SUM_BAD_DEBT,BR_MONTHS_STATUS_C_SUM_CLOSED,BR_MONTHS_STATUS_C_SUM_SOLD,BR_MONTHS_STATUS_X_SUM_ACTIVE,BR_MONTHS_STATUS_X_SUM_BAD_DEBT,BR_MONTHS_STATUS_X_SUM_CLOSED,BR_MONTHS_STATUS_X_SUM_SOLD,BR_MONTHS_COUNT_SUM_ACTIVE,BR_MONTHS_COUNT_SUM_BAD_DEBT,BR_MONTHS_COUNT_SUM_CLOSED,BR_MONTHS_COUNT_SUM_SOLD,BR_MONTHS_MAX_MAX_ACTIVE,BR_MONTHS_MAX_MAX_BAD_DEBT,BR_MONTHS_MAX_MAX_CLOSED,BR_MONTHS_MAX_MAX_SOLD,BR_MONTHS_MIN_MIN_ACTIVE,BR_MONTHS_MIN_MIN_BAD_DEBT,BR_MONTHS_MIN_MIN_CLOSED,BR_MONTHS_MIN_MIN_SOLD,BR_SK_ID_BUREAU_COUNT_ACTIVE,BR_SK_ID_BUREAU_COUNT_BAD_DEBT,BR_SK_I

In [36]:
describe_df(df_bureau_type)

(305436, 4)


,BR_TYPE_COUNT_SK_ID_BUREAU_CAR_LOAN,BR_TYPE_COUNT_SK_ID_BUREAU_CONSUMER_CREDIT,BR_TYPE_COUNT_SK_ID_BUREAU_CREDIT_CARD,BR_TYPE_COUNT_SK_ID_BUREAU_MORTGAGE
count,305436.0000,305436.0000,305436.0000,305436.0000
mean,0.0907,4.0978,1.3168,0.0602
std,0.3514,3.5338,1.5130,0.2619
min,0.0000,0.0000,0.0000,0.0000
25%,0.0000,2.0000,0.0000,0.0000
50%,0.0000,3.0000,1.0000,0.0000
75%,0.0000,6.0000,2.0000,0.0000
max,10.0000,86.0000,22.0000,13.0000


### 2.2) Previous application-related features

In [24]:
df_prev_app = pd.read_csv(data_path + 'previous_application.csv')
df_prev_app

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_TYPE_SUITE,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,SELLERPLACE_AREA,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495,271877,Consumer loans,1730.4300,17145.0000,17145.0000,0.0000,17145.0000,SATURDAY,15,Y,1,0.0000,0.1828,0.8673,XAP,Approved,-73,Cash through the bank,XAP,NaN,Repeater,Mobile,POS,XNA,Country-wide,35,Connectivity,12.0000,middle,POS mobile with interest,365243.0000,-42.0000,300.0000,-42.0000,-37.0000,0.0000
1,2802425,108129,Cash loans,25188.6150,607500.0000,679671.0000,NaN,607500.0000,THURSDAY,11,Y,1,NaN,NaN,NaN,XNA,Approved,-164,XNA,XAP,Unaccompanied,Repeater,XNA,Cash,x-sell,Contact center,-1,XNA,36.0000,low_action,Cash X-Sell: low,365243.0000,-134.0000,916.0000,365243.0000,365243.0000,1.0000
2,2523466,122040,Cash loans,15060.7350,112500.0000,136444.5000,NaN,112500.0000,TUESDAY,11,Y,1,NaN,NaN,NaN,XNA,Approved,-301,Cash through the bank,XAP,"Spouse, partner",Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,12.0000,high,Cash X-Sell: high,365243.0000,-271.0000,59.0000,365243.0000,365243.0000,1.0000
3,2819243,176158,Cash loans,47041.3350,450000.0000,470790.0000,NaN,450000.0000,MONDAY,7,Y,1,NaN,NaN,NaN,XNA,Approved,-512,Cash through the bank,XAP,NaN,Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,12.0000,middle,Cash X-Sell: middle,365243.0000,-482.0000,-152.0000,-182.0000,-177.0000,1.0000
4,1784265,202054,Cash loans,31924.3950,337500.0000,404055.0000,NaN,337500.0000,THURSDAY,9,Y,1,NaN,NaN,NaN,Repairs,Refused,-781,Cash through the bank,HC,NaN,Repeater,XNA,Cash,walk-in,Credit and cash offices,-1,XNA,24.0000,high,Cash Street: high,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1670209,2300464,352015,Consumer loans,14704.2900,267295.5000,311400.0000,0.0000,267295.5000,WEDNESDAY,12,Y,1,0.0000,NaN,NaN,XAP,Approved,-544,Cash through the bank,XAP,NaN,Refreshed,Furniture,POS,XNA,Stone,43,Furniture,30.0000,low_normal,POS industry with interest,365243.0000,-508.0000,362.0000,-358.0000,-351.0000,0.0000
1670210,2357031,334635,Consumer loans,6622.0200,87750.0000,64291.5000,29250.0000,87750.0000,TUESDAY,15,Y,1,0.3406,NaN,NaN,XAP,Approved,-1694,Cash through the bank,XAP,Unaccompanied,New,Furniture,POS,XNA,Stone,43,Furniture,12.0000,middle,POS industry with interest,365243.0000,-1604.0000,-1274.0000,-1304.0000,-1297.0000,0.0000
1670211,2659632,249544,Consumer loans,11520.8550,105237.0000,102523.5000,10525.5000,105237.0000,MONDAY,12,Y,1,0.1014,NaN,NaN,XAP,Approved,-1488,Cash through the bank,XAP,"Spouse, partner",Repeater,Consumer Electronics,POS,XNA,Country-wide,1370,Consumer electronics,10.0000,low_normal,POS household with interest,365243.0000,-1457.0000,-1187.0000,-1187.0000,-1181.0000,0.0000
1670212,2785582,400317,Cash loans,18821.5200,180000.0000,191880.0000,NaN,180000.0000,WEDNESDAY,9,Y,1,NaN,NaN,NaN,XNA,Approved,-1185,Cash through the bank,XAP,Family,Repeater,XNA,Cash,x-sell,AP+ (Cash loan),-1,XNA,12.0000,low_normal,Cash X-Sell: low,365243.0000,-1155.0000,-825.0000,-825.0000,-817.0000,1.0000


In [25]:
df_pos_bal = pd.read_csv(data_path + 'POS_CASH_balance.csv')
df_pos_bal['FLAG_COMPLETED'] = (df_pos_bal['NAME_CONTRACT_STATUS'] == 'Completed').astype('int')
df_pos_bal

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF,FLAG_COMPLETED
0,1803195,182943,-31,48.0000,45.0000,Active,0,0,0
1,1715348,367990,-33,36.0000,35.0000,Active,0,0,0
2,1784872,397406,-32,12.0000,9.0000,Active,0,0,0
3,1903291,269225,-35,48.0000,42.0000,Active,0,0,0
4,2341044,334279,-35,36.0000,35.0000,Active,0,0,0
...,...,...,...,...,...,...,...,...,...
10001353,2448283,226558,-20,6.0000,0.0000,Active,843,0,0
10001354,1717234,141565,-19,12.0000,0.0000,Active,602,0,0
10001355,1283126,315695,-21,10.0000,0.0000,Active,609,0,0
10001356,1082516,450255,-22,12.0000,0.0000,Active,614,0,0


In [28]:
df_credit_bal = pd.read_csv(data_path + 'credit_card_balance.csv')
df_credit_bal['FLAG_COMPLETED'] = (df_credit_bal['NAME_CONTRACT_STATUS'] == 'Completed').astype('int')
df_credit_bal

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_CURRENT,AMT_PAYMENT_TOTAL_CURRENT,AMT_RECEIVABLE_PRINCIPAL,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF,FLAG_COMPLETED
0,2562384,378907,-6,56.9700,135000,0.0000,877.5000,0.0000,877.5000,1700.3250,1800.0000,1800.0000,0.0000,0.0000,0.0000,0.0000,1,0.0000,1.0000,35.0000,Active,0,0,0
1,2582071,363914,-1,63975.5550,45000,2250.0000,2250.0000,0.0000,0.0000,2250.0000,2250.0000,2250.0000,60175.0800,64875.5550,64875.5550,1.0000,1,0.0000,0.0000,69.0000,Active,0,0,0
2,1740877,371185,-7,31815.2250,450000,0.0000,0.0000,0.0000,0.0000,2250.0000,2250.0000,2250.0000,26926.4250,31460.0850,31460.0850,0.0000,0,0.0000,0.0000,30.0000,Active,0,0,0
3,1389973,337855,-4,236572.1100,225000,2250.0000,2250.0000,0.0000,0.0000,11795.7600,11925.0000,11925.0000,224949.2850,233048.9700,233048.9700,1.0000,1,0.0000,0.0000,10.0000,Active,0,0,0
4,1891521,126868,-1,453919.4550,450000,0.0000,11547.0000,0.0000,11547.0000,22924.8900,27000.0000,27000.0000,443044.3950,453919.4550,453919.4550,0.0000,1,0.0000,1.0000,101.0000,Active,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3840307,1036507,328243,-9,0.0000,45000,NaN,0.0000,NaN,NaN,0.0000,NaN,0.0000,0.0000,0.0000,0.0000,NaN,0,NaN,NaN,0.0000,Active,0,0,0
3840308,1714892,347207,-9,0.0000,45000,0.0000,0.0000,0.0000,0.0000,0.0000,1879.1100,0.0000,0.0000,0.0000,0.0000,0.0000,0,0.0000,0.0000,23.0000,Active,0,0,0
3840309,1302323,215757,-9,275784.9750,585000,270000.0000,270000.0000,0.0000,0.0000,2250.0000,375750.0000,356994.6750,269356.1400,273093.9750,273093.9750,2.0000,2,0.0000,0.0000,18.0000,Active,0,0,0
3840310,1624872,430337,-10,0.0000,450000,NaN,0.0000,NaN,NaN,0.0000,NaN,0.0000,0.0000,0.0000,0.0000,NaN,0,NaN,NaN,0.0000,Active,0,0,0


In [27]:
df_pos_agg = (
    df_pos_bal
    .groupby('SK_ID_PREV')
    .agg({
        'MONTHS_BALANCE': ['max', 'min', 'count'],
        'CNT_INSTALMENT': ['max', 'mean'],
        'CNT_INSTALMENT_FUTURE': 'min',
        'SK_DPD': ['sum', 'max', 'mean'],
        'SK_DPD_DEF': ['sum', 'max', 'mean'],
        'FLAG_COMPLETED': ['max']
    })
)
df_pos_agg.columns = clean_columns_name(df_pos_agg, 'POS_')

In [29]:
amt_aggs = {col: ['sum', 'max', 'mean'] for col in df_credit_bal.columns if col[:4] in ('AMT_', 'CNT_', 'SK_D')}
df_credit_agg = (
    df_credit_bal
    .groupby('SK_ID_PREV')
    .agg({
        'MONTHS_BALANCE': ['max', 'min', 'count'],
        'FLAG_COMPLETED': ['max'],
        **amt_aggs
    })
)
df_credit_agg.columns = clean_columns_name(df_credit_agg, 'CR_')

In [31]:
df_prev_agg = (
    df_prev_app
    .merge(df_pos_agg, on='SK_ID_PREV', how='left')
    .merge(df_credit_agg, on='SK_ID_PREV', how='left')
)
df_prev_agg['CREDIT_TO_APP'] = df_prev_agg['AMT_CREDIT'] - df_prev_agg['AMT_APPLICATION']
df_prev_agg['PCT_CREDIT_TO_APP'] = df_prev_agg['AMT_CREDIT'] / df_prev_agg['AMT_APPLICATION'] - 1

In [32]:
df_prev_type = (
    df_prev_agg
    .pivot_table(
        index='SK_ID_CURR',
        columns='NAME_CONTRACT_TYPE',
        aggfunc={
            'SK_ID_PREV': 'count',
            'AMT_ANNUITY': ['sum', 'max', 'mean'],
            'AMT_APPLICATION': ['sum', 'max', 'mean'],
            'AMT_CREDIT': ['sum', 'max', 'mean'],
            'CREDIT_TO_APP': ['sum', 'max', 'min'],
            'PCT_CREDIT_TO_APP': ['max', 'min', 'mean'],
            'RATE_DOWN_PAYMENT': ['mean'],
            'RATE_INTEREST_PRIMARY': ['max', 'min', 'mean'],
            'RATE_INTEREST_PRIVILEGED': ['max', 'min', 'mean'],
            'DAYS_DECISION': ['max', 'min'],
            **{c: 'max' for c in df_prev_agg.columns if '_max' in c},
            **{c: 'min' for c in df_prev_agg.columns if '_min' in c},
            **{c: 'mean' for c in df_prev_agg.columns if '_mean' in c},
            **{c: 'sum' for c in df_prev_agg.columns if '_sum' in c or '_count' in c},
        },
        fill_value=0
    )
)
df_prev_type.columns = clean_columns_name(df_prev_type, 'PREV_')

df_prev_status = (
    df_prev_app
    .pivot_table(
        index='SK_ID_CURR',
        columns='NAME_CONTRACT_STATUS',
        values=['SK_ID_PREV'],
        aggfunc=['count'],
        fill_value=0
    )
)
df_prev_status.columns = [col[2] for col in df_prev_status.columns]
prev_count = df_prev_status.sum(axis=1)
for col in df_prev_status.columns:
    df_prev_status['PCT_'+ col] = df_prev_status[col]/prev_count
df_prev_status.columns = clean_columns_name(df_prev_status, 'PREV_STATUS_')

In [37]:
describe_df(df_prev_type)

(338857, 100)


,PREV_AMT_ANNUITY_MAX_CASH_LOANS,PREV_AMT_ANNUITY_MAX_CONSUMER_LOANS,PREV_AMT_ANNUITY_MAX_REVOLVING_LOANS,PREV_AMT_ANNUITY_MAX_XNA,PREV_AMT_ANNUITY_MEAN_CASH_LOANS,PREV_AMT_ANNUITY_MEAN_CONSUMER_LOANS,PREV_AMT_ANNUITY_MEAN_REVOLVING_LOANS,PREV_AMT_ANNUITY_MEAN_XNA,PREV_AMT_ANNUITY_SUM_CASH_LOANS,PREV_AMT_ANNUITY_SUM_CONSUMER_LOANS,PREV_AMT_ANNUITY_SUM_REVOLVING_LOANS,PREV_AMT_ANNUITY_SUM_XNA,PREV_AMT_APPLICATION_MAX_CASH_LOANS,PREV_AMT_APPLICATION_MAX_CONSUMER_LOANS,PREV_AMT_APPLICATION_MAX_REVOLVING_LOANS,PREV_AMT_APPLICATION_MAX_XNA,PREV_AMT_APPLICATION_MEAN_CASH_LOANS,PREV_AMT_APPLICATION_MEAN_CONSUMER_LOANS,PREV_AMT_APPLICATION_MEAN_REVOLVING_LOANS,PREV_AMT_APPLICATION_MEAN_XNA,PREV_AMT_APPLICATION_SUM_CASH_LOANS,PREV_AMT_APPLICATION_SUM_CONSUMER_LOANS,PREV_AMT_APPLICATION_SUM_REVOLVING_LOANS,PREV_AMT_APPLICATION_SUM_XNA,PREV_AMT_CREDIT_MAX_CASH_LOANS,PREV_AMT_CREDIT_MAX_CONSUMER_LOANS,PREV_AMT_CREDIT_MAX_REVOLVING_LOANS,PREV_AMT_CREDIT_MAX_XNA,PREV_AMT_CREDIT_MEAN_CASH_LOANS,PREV_AMT_CREDIT_MEAN_CONSUMER_LOANS,PREV_AMT_CREDIT_MEAN_REVOLVING_LOANS,PREV_AMT_CREDIT_MEAN_XNA,PREV_AMT_CREDIT_SUM_CASH_LOANS,PREV_AMT_CREDIT_SUM_CONSUMER_LOANS,PREV_AMT_CREDIT_SUM_REVOLVING_LOANS,PREV_AMT_CREDIT_SUM_XNA,PREV_CREDIT_TO_APP_MAX_CASH_LOANS,PREV_CREDIT_TO_APP_MAX_CONSUMER_LOANS,PREV_CREDIT_TO_APP_MAX_REVOLVING_LOANS,PREV_CREDIT_TO_APP_MAX_XNA,PREV_CREDIT_TO_APP_MIN_CASH_LOANS,PREV_CREDIT_TO_APP_MIN_CONSUMER_LOANS,PREV_CREDIT_TO_APP_MIN_REVOLVING_LOANS,PREV_CREDIT_TO_APP_MIN_XNA,PREV_CREDIT_TO_APP_SUM_CASH_LOANS,PREV_CREDIT_TO_APP_SUM_CONSUMER_LOANS,PREV_CREDIT_TO_APP_SUM_REVOLVING_LOANS,PREV_CREDIT_TO_APP_SUM_XNA,PREV_DAYS_DECISION_MAX_CASH_LOANS,PREV_DAYS_DECISION_MAX_CONSUMER_LOANS,PREV_DAYS_DECISION_MAX_REVOLVING_LOANS,PREV_DAYS_DECISION_MAX_XNA,PREV_DAYS_DECISION_MIN_CASH_LOANS,PREV_DAYS_DECISION_MIN_CONSUMER_LOANS,PREV_DAYS_DECISION_MIN_REVOLVING_LOANS,PREV_DAYS_DECISION_MIN_XNA,PREV_PCT_CREDIT_TO_APP_MAX_CASH_LOANS,PREV_PCT_CREDIT_TO_APP_MAX_CONSUMER_LOANS,PREV_PCT_CREDIT_TO_APP_MAX_REVOLVING_LOANS,PREV_PCT_CREDIT_TO_APP_MAX_XNA,PREV_PCT_CREDIT_TO_APP_MEAN_CASH_LOANS,PREV_PCT_CREDIT_TO_APP_MEAN_CONSUMER_LOANS,PREV_PCT_CREDIT_TO_APP_MEAN_REVOLVING_LOANS,PREV_PCT_CREDIT_TO_APP_MEAN_XNA,PREV_PCT_CREDIT_TO_APP_MIN_CASH_LOANS,PREV_PCT_CREDIT_TO_APP_MIN_CONSUMER_LOANS,PREV_PCT_CREDIT_TO_APP_MIN_REVOLVING_LOANS,PREV_PCT_CREDIT_TO_APP_MIN_XNA,PREV_RATE_DOWN_PAYMENT_MEAN_CASH_LOANS,PREV_RATE_DOWN_PAYMENT_MEAN_CONSUMER_LOANS,PREV_RATE_DOWN_PAYMENT_MEAN_REVOLVING_LOANS,PREV_RATE_DOWN_PAYMENT_MEAN_XNA,PREV_RATE_INTEREST_PRIMARY_MAX_CASH_LOANS,PREV_RATE_INTEREST_PRIMARY_MAX_CONSUMER_LOANS,PREV_RATE_INTEREST_PRIMARY_MAX_REVOLVING_LOANS,PREV_RATE_INTEREST_PRIMARY_MAX_XNA,PREV_RATE_INTEREST_PRIMARY_MEAN_CASH_LOANS,PREV_RATE_INTEREST_PRIMARY_MEAN_CONSUMER_LOANS,PREV_RATE_INTEREST_PRIMARY_MEAN_REVOLVING_LOANS,PREV_RATE_INTEREST_PRIMARY_MEAN_XNA,PREV_RATE_INTEREST_PRIMARY_MIN_CASH_LOANS,PREV_RATE_INTEREST_PRIMARY_MIN_CONSUMER_LOANS,PREV_RATE_INTEREST_PRIMARY_MIN_REVOLVING_LOANS,PREV_RATE_INTEREST_PRIMARY_MIN_XNA,PREV_RATE_INTEREST_PRIVILEGED_MAX_CASH_LOANS,PREV_RATE_INTEREST_PRIVILEGED_MAX_CONSUMER_LOANS,PREV_RATE_INTEREST_PRIVILEGED_MAX_REVOLVING_LOANS,PREV_RATE_INTEREST_PRIVILEGED_MAX_XNA,PREV_RATE_INTEREST_PRIVILEGED_MEAN_CASH_LOANS,PREV_RATE_INTEREST_PRIVILEGED_MEAN_CONSUMER_LOANS,PREV_RATE_INTEREST_PRIVILEGED_MEAN_REVOLVING_LOANS,PREV_RATE_INTEREST_PRIVILEGED_MEAN_XNA,PREV_RATE_INTEREST_PRIVILEGED_MIN_CASH_LOANS,PREV_RATE_INTEREST_PRIVILEGED_MIN_CONSUMER_LOANS,PREV_RATE_INTEREST_PRIVILEGED_MIN_REVOLVING_LOANS,PREV_RATE_INTEREST_PRIVILEGED_MIN_XNA,PREV_SK_ID_PREV_COUNT_CASH_LOANS,PREV_SK_ID_PREV_COUNT_CONSUMER_LOANS,PREV_SK_ID_PREV_COUNT_REVOLVING_LOANS,PREV_SK_ID_PREV_COUNT_XNA
count,338857.0000,338857.0000,338857.0000,338857.0000,338857.0000,338857.0000,338857.0000,338857.0000,338857.0000,338857.0000,338857.0000,338857.0000,338857.0000,338857.0000,338857.0000,338857.0000,338857.0000,338857.0000,338857.0000,338857.0000,338857.0000,338857.0000

In [38]:
describe_df(df_prev_status)

(338857, 8)


,PREV_STATUS_A_P_P_R_O_V_E_D,PREV_STATUS_C_A_N_C_E_L_E_D,PREV_STATUS_R_E_F_U_S_E_D,PREV_STATUS_U_N_U_S_E_D___O_F_F_E_R,PREV_STATUS_P_C_T___A_P_P_R_O_V_E_D,PREV_STATUS_P_C_T___C_A_N_C_E_L_E_D,PREV_STATUS_P_C_T___R_E_F_U_S_E_D,PREV_STATUS_P_C_T___U_N_U_S_E_D___O_F_F_E_R
count,338857.0000,338857.0000,338857.0000,338857.0000,338857.0000,338857.0000,338857.0000,338857.0000
mean,3.0596,0.9335,0.8578,0.0780,0.7445,0.1292,0.1114,0.0149
std,2.1354,1.7230,1.8306,0.3280,0.2632,0.1887,0.1837,0.0686
min,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
25%,1.0000,0.0000,0.0000,0.0000,0.5000,0.0000,0.0000,0.0000
50%,3.0000,0.0000,0.0000,0.0000,0.7778,0.0000,0.0000,0.0000
75%,4.0000,1.0000,1.0000,0.0000,1.0000,0.2500,0.2000,0.0000
max,27.0000,58.0000,68.0000,21.0000,1.0000,1.0000,1.0000,1.0000


## Step 3: Feature Selection

Due to runtime constraints, we apologize for not showing the full feature selection process.  Below is a brief summary of the final feature selection steps. To manage memory usage, we limited the feature set size and re-train the model using various combinations of top-ranked features based on SHAP values.

1. Trained on application features only and selected the top application features.
2. Trained on the top application features combined with the full set of bureau features, then selected the top bureau features.
3. Trained on the top application features combined with the full set of previous application features, then selected the top previous application features.
4. Trained on the combined top features from application, bureau, and previous application data, then selected the overall top features.
5. Retained only the aggregate functions that generated the top features for the final model.

## Step 4: Missing value handling
We explored on below strategies
1. Filling missing values with zero and mean
2. Filling all missing values with zero
4. No filling (leave missing)
5. Filling missing values only with zero (this yielded the best results)